# Change Tracker - Database Explorer

## 🎯 Purpose

This notebook provides a comprehensive view of **all data** in the FAQ update tracking system:
- **Browse All Tables** - View contents of every table
- **Explore VIEWs** - See derived data and relationships
- **Search & Filter** - Find specific records
- **Export Data** - Extract data for external analysis
- **Understand Structure** - See table schemas and relationships

### ✨ Features

1. **Complete Table Browser** - Every table with configurable row limits
2. **VIEW Explorer** - All 9 VIEWs with sample data
3. **Schema Inspector** - Column definitions and data types
4. **Relationship Mapper** - Foreign key relationships
5. **Search Functionality** - Find records across tables
6. **Data Export** - Save to CSV for external analysis
7. **Interactive Filters** - Customize what you see

### ⚠️ Prerequisites

- Database must exist: `../databases/faq_update.db`
- Run `setup_tables.ipynb` first if database doesn't exist

## 📖 When to Run

- Exploring database contents
- Understanding data structure
- Finding specific records
- Debugging data issues
- Learning the system

## Step 1: Configuration and Setup

In [23]:
import sys
import sqlite3
import pandas as pd
from pathlib import Path
from datetime import datetime
import json

# ============================================================================
# 📁 PATH CONFIGURATION
# ============================================================================

NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent
SRC_DIR = PROJECT_ROOT / 'src'

# Add src to Python path for utility imports
sys.path.insert(0, str(SRC_DIR))

PROJECT_ROOT = NOTEBOOK_DIR.parent
DATABASE_DIR = PROJECT_ROOT / 'databases'
DATABASE_NAME = 'faq_update.db'
DB_PATH = DATABASE_DIR / DATABASE_NAME
EXPORT_DIR = PROJECT_ROOT / 'exports'

# Create export directory if needed
EXPORT_DIR.mkdir(parents=True, exist_ok=True)

# ============================================================================
# ⚙️ DISPLAY SETTINGS
# ============================================================================

# Configure display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 50)

# Configurable limits (adjust these as needed)
DEFAULT_ROW_LIMIT = 20      # Default rows to display per table
VIEW_ROW_LIMIT = 15         # Default rows for VIEWs
SEARCH_LIMIT = 50           # Max search results

# ============================================================================
# 🎨 FORMATTING FUNCTIONS
# ============================================================================

def print_header(title: str, level: int = 1, char: str = None):
    """Print formatted header."""
    chars = ['═', '─', '·', '-']
    if char:
        symbol = char
    else:
        symbol = chars[min(level - 1, len(chars) - 1)]
    width = 120
    print(f"\n{symbol * width}")
    print(f"{title.center(width)}")
    print(f"{symbol * width}\n")

def print_table_info(table_name: str, count: int, description: str = ""):
    """Print table information."""
    print(f"\n📊 Table: {table_name}")
    if description:
        print(f"   Description: {description}")
    print(f"   Records: {count:,}")
    print("-" * 120)

def print_view_info(view_name: str, description: str = ""):
    """Print VIEW information."""
    print(f"\n👁️  View: {view_name}")
    if description:
        print(f"   Description: {description}")
    print("-" * 120)

def display_dataframe(df: pd.DataFrame, max_rows: int = None):
    """Display DataFrame with formatting."""
    if len(df) == 0:
        print("   (No records found)\n")
        return
    
    display_rows = max_rows if max_rows else DEFAULT_ROW_LIMIT
    
    if len(df) <= display_rows:
        print(df.to_string(index=False))
        print(f"\n   Showing all {len(df)} rows")
    else:
        print(df.head(display_rows).to_string(index=False))
        print(f"\n   Showing first {display_rows} of {len(df):,} rows")
        print(f"   (Adjust DEFAULT_ROW_LIMIT in configuration to see more)")
    print()

def print_schema(conn: sqlite3.Connection, table_name: str):
    """Print table schema."""
    schema = pd.read_sql_query(f"PRAGMA table_info({table_name})", conn)
    print("\n   Schema:")
    for _, row in schema.iterrows():
        pk = " [PRIMARY KEY]" if row['pk'] == 1 else ""
        notnull = " NOT NULL" if row['notnull'] == 1 else ""
        print(f"      • {row['name']:30} {row['type']:15} {notnull}{pk}")
    print()

def export_to_csv(df: pd.DataFrame, filename: str):
    """Export DataFrame to CSV."""
    filepath = EXPORT_DIR / filename
    df.to_csv(filepath, index=False)
    print(f"   ✅ Exported to: {filepath}")

print("=" * 120)
print("🔍 CHANGE TRACKER - Database Explorer".center(120))
print("=" * 120)
print(f"\n📁 Database: {DB_PATH.absolute()}")
print(f"📁 Export Directory: {EXPORT_DIR.absolute()}")
print(f"⏰ Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"\n⚙️  Display Settings:")
print(f"   • Default Row Limit: {DEFAULT_ROW_LIMIT}")
print(f"   • View Row Limit: {VIEW_ROW_LIMIT}")
print(f"   • Search Limit: {SEARCH_LIMIT}")
print("=" * 120)

print("\n✅ Configuration loaded")

                                          🔍 CHANGE TRACKER - Database Explorer                                          

📁 Database: c:\Users\praka\projects\FAQ\FAQ_update\databases\faq_update.db
📁 Export Directory: c:\Users\praka\projects\FAQ\FAQ_update\exports
⏰ Timestamp: 2025-10-07 07:04:25

⚙️  Display Settings:
   • Default Row Limit: 20
   • View Row Limit: 15
   • Search Limit: 50

✅ Configuration loaded


## Step 2: Database Overview

In [24]:
print_header("DATABASE OVERVIEW", level=1)

# Verify database exists
if not DB_PATH.exists():
    print(f"❌ Database not found: {DB_PATH}")
    print("\n⚠️  Please run setup_tables.ipynb first to create the database")
    raise FileNotFoundError(f"Database file not found: {DB_PATH}")

print(f"✅ Database found: {DB_PATH.name}\n")

with sqlite3.connect(DB_PATH) as conn:
    # Enable foreign keys
    conn.execute("PRAGMA foreign_keys = ON")
    
    # Get database info
    import os
    db_size = os.path.getsize(DB_PATH)
    print(f"💾 Database Size: {db_size:,} bytes ({db_size/1024/1024:.2f} MB)")
    
    version = conn.execute("SELECT sqlite_version()").fetchone()[0]
    print(f"🔧 SQLite Version: {version}")
    
    # Get all objects
    all_objects = pd.read_sql_query("""
        SELECT 
            type,
            name,
            sql
        FROM sqlite_master 
        WHERE type IN ('table', 'view', 'index', 'trigger')
          AND name NOT LIKE 'sqlite_%'
        ORDER BY type, name
    """, conn)
    
    # Count by type
    type_counts = all_objects['type'].value_counts()
    
    print(f"\n📊 Database Objects:")
    for obj_type, count in type_counts.items():
        print(f"   • {obj_type.capitalize()}s: {count}")
    
    # List all tables with row counts
    print(f"\n\n📋 BASE TABLES:")
    print("-" * 120)
    
    tables = all_objects[all_objects['type'] == 'table']['name'].tolist()
    table_stats = []
    
    for table in tables:
        count = conn.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
        table_stats.append({'Table': table, 'Records': count})
    
    table_df = pd.DataFrame(table_stats)
    if len(table_df) > 0:
        table_df = table_df.sort_values('Records', ascending=False)
        print(table_df.to_string(index=False))
    
    # List all views
    print(f"\n\n👁️  VIEWS:")
    print("-" * 120)
    
    views = all_objects[all_objects['type'] == 'view']['name'].tolist()
    for view in sorted(views):
        print(f"   • {view}")
    
    # List indexes
    print(f"\n\n🔍 INDEXES:")
    print("-" * 120)
    
    indexes = all_objects[all_objects['type'] == 'index']['name'].tolist()
    print(f"   Total Indexes: {len(indexes)}")
    if len(indexes) <= 20:
        for idx in sorted(indexes):
            print(f"   • {idx}")
    else:
        print(f"   (Too many to list - {len(indexes)} indexes present)")

print("\n✅ Database overview complete")


════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════
                                                   DATABASE OVERVIEW                                                    
════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════

✅ Database found: faq_update.db

💾 Database Size: 217,088 bytes (0.21 MB)
🔧 SQLite Version: 3.45.1

📊 Database Objects:
   • Indexs: 35
   • Views: 9
   • Tables: 8
   • Triggers: 1


📋 BASE TABLES:
------------------------------------------------------------------------------------------------------------------------
                       Table  Records
               faq_questions       10
                 faq_answers       10
          content_change_log        9
                content_repo        6
content_repo_test_operations        3
              schema_version        1
               content_diffs        0
           

## Step 3: BASE TABLES - Content Repository

### 3.1 content_repo

**Purpose**: Source content repository containing all documents, pages, and markdown extracts.

**Key Fields**:
- `ud_source_file_id` - Primary key (auto-increment)
- `raw_file_nme` - Source file name
- `raw_file_page_nbr` - Page number within file
- `content_checksum` - SHA-256 hash of content
- `version_nbr` - Version number of this page
- `file_status` - Active, Inactive, or Archived

In [25]:
print_header("TABLE: content_repo", level=2)

with sqlite3.connect(DB_PATH) as conn:
    # Get count
    count = conn.execute("SELECT COUNT(*) FROM content_repo").fetchone()[0]
    
    print_table_info(
        "content_repo",
        count,
        "Source content repository - all documents and pages"
    )
    
    # Show schema
    print_schema(conn, "content_repo")
    
    # Load data
    if count > 0:
        df = pd.read_sql_query(f"""
            SELECT 
                ud_source_file_id,
                domain,
                service,
                raw_file_nme,
                raw_file_page_nbr,
                version_nbr,
                content_checksum,
                file_status,
                created_dt,
                last_modified_dt
            FROM content_repo
            ORDER BY ud_source_file_id DESC
            LIMIT {DEFAULT_ROW_LIMIT * 2}
        """, conn)
        
        print("\n📄 Recent Records:")
        display_dataframe(df)
        
        # Show statistics
        print("\n📊 Statistics:")
        stats = pd.read_sql_query("""
            SELECT 
                COUNT(*) as total_pages,
                COUNT(DISTINCT raw_file_nme) as unique_files,
                COUNT(DISTINCT content_checksum) as unique_checksums,
                AVG(version_nbr) as avg_version,
                MAX(version_nbr) as max_version,
                SUM(CASE WHEN file_status = 'Active' THEN 1 ELSE 0 END) as active_pages,
                SUM(CASE WHEN file_status = 'Inactive' THEN 1 ELSE 0 END) as inactive_pages
            FROM content_repo
        """, conn).iloc[0]
        
        for key, value in stats.items():
            if isinstance(value, float):
                print(f"   • {key}: {value:.2f}")
            else:
                print(f"   • {key}: {value:,}")
        
        # Files breakdown
        print("\n\n📂 Files Breakdown:")
        files_df = pd.read_sql_query("""
            SELECT 
                raw_file_nme as file_name,
                COUNT(*) as pages,
                MAX(version_nbr) as max_version,
                MAX(last_modified_dt) as last_modified
            FROM content_repo
            GROUP BY raw_file_nme
            ORDER BY pages DESC
        """, conn)
        display_dataframe(files_df, max_rows=15)
    else:
        print("\n   ⚠️  No records in content_repo")

print("✅ content_repo explored")


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                  TABLE: content_repo                                                   
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


📊 Table: content_repo
   Description: Source content repository - all documents and pages
   Records: 6
------------------------------------------------------------------------------------------------------------------------

   Schema:
      • ud_source_file_id              INTEGER          [PRIMARY KEY]
      • domain                         TEXT            
      • service                        TEXT            
      • orgoid                         TEXT            
      • associateoid                   TEXT            
      • raw_file_nme                   TEXT             NOT NULL
      • raw_file_type                 

## Step 4: BASE TABLES - FAQ Data

### 4.1 faq_questions

**Purpose**: FAQ questions master table with version history.

**Key Fields**:
- `question_id` - Primary key
- `question_txt` - Current question text
- `prev_question_txt` - Previous version
- `source` - Origin of the question
- `domain`, `service`, `product` - Organization context

In [26]:
print_header("TABLE: faq_questions", level=2)

with sqlite3.connect(DB_PATH) as conn:
    count = conn.execute("SELECT COUNT(*) FROM faq_questions").fetchone()[0]
    
    print_table_info(
        "faq_questions",
        count,
        "FAQ questions with version history"
    )
    
    print_schema(conn, "faq_questions")
    
    if count > 0:
        df = pd.read_sql_query(f"""
            SELECT 
                question_id,
                SUBSTR(question_txt, 1, 100) as question_preview,
                source,
                domain,
                service,
                status,
                created,
                modified
            FROM faq_questions
            ORDER BY question_id DESC
            LIMIT {DEFAULT_ROW_LIMIT}
        """, conn)
        
        print("\n📄 Recent Questions:")
        display_dataframe(df)
        
        # Statistics
        print("\n📊 Statistics:")
        stats = pd.read_sql_query("""
            SELECT 
                COUNT(*) as total_questions,
                COUNT(DISTINCT domain) as domains,
                COUNT(DISTINCT service) as services,
                COUNT(DISTINCT source) as sources,
                SUM(CASE WHEN prev_question_txt IS NOT NULL THEN 1 ELSE 0 END) as questions_with_history
            FROM faq_questions
        """, conn).iloc[0]
        
        for key, value in stats.items():
            print(f"   • {key}: {value:,}")
        
        # Breakdown by domain/service
        print("\n\n📊 Questions by Domain/Service:")
        breakdown = pd.read_sql_query("""
            SELECT 
                domain,
                service,
                COUNT(*) as question_count
            FROM faq_questions
            GROUP BY domain, service
            ORDER BY question_count DESC
        """, conn)
        display_dataframe(breakdown, max_rows=10)
    else:
        print("\n   ⚠️  No records in faq_questions")

print("✅ faq_questions explored")


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                  TABLE: faq_questions                                                  
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


📊 Table: faq_questions
   Description: FAQ questions with version history
   Records: 10
------------------------------------------------------------------------------------------------------------------------

   Schema:
      • question_id                    INTEGER          [PRIMARY KEY]
      • prev_recommended_question_txt  TEXT            
      • prev_question_txt              TEXT            
      • recommeded_question_txt        TEXT            
      • question_txt                   TEXT             NOT NULL
      • source                         TEXT            
      • src_file_name                  TEXT          

### 4.2 faq_answers

**Purpose**: FAQ answers linked to questions with version history.

**Key Fields**:
- `answer_id` - Primary key
- `question_id` - Foreign key to faq_questions
- `faq_answer_txt` - Current answer text
- `prev_faq_answer_txt` - Previous version
- `user_feedback_txt` - User feedback on answer

In [48]:
print_header("TABLE: faq_answers", level=2)

with sqlite3.connect(DB_PATH) as conn:
    count = conn.execute("SELECT COUNT(*) FROM faq_answers").fetchone()[0]
    
    print_table_info(
        "faq_answers",
        count,
        "FAQ answers linked to questions with version history"
    )
    
    print_schema(conn, "faq_answers")
    
    if count > 0:
        df = pd.read_sql_query(f"""
            SELECT 
                answer_id,
                question_id,
                SUBSTR(faq_answer_txt, 1, 100) as answer_preview,
                source,
                domain,
                service,
                status,
                created,
                modified
            FROM faq_answers
            ORDER BY answer_id DESC
            LIMIT {DEFAULT_ROW_LIMIT}
        """, conn)
        
        print("\n📄 Recent Answers:")
        display_dataframe(df)
        
        # Statistics
        print("\n📊 Statistics:")
        stats = pd.read_sql_query("""
            SELECT 
                COUNT(*) as total_answers,
                COUNT(DISTINCT question_id) as unique_questions,
                SUM(CASE WHEN prev_faq_answer_txt IS NOT NULL THEN 1 ELSE 0 END) as answers_with_history,
                SUM(CASE WHEN user_feedback_txt IS NOT NULL THEN 1 ELSE 0 END) as answers_with_feedback
            FROM faq_answers
        """, conn).iloc[0]
        
        for key, value in stats.items():
            print(f"   • {key}: {value:,}")
        
        # Q&A pairing check
        print("\n\n🔗 Question-Answer Pairing:")
        pairing = pd.read_sql_query("""
            SELECT 
                (SELECT COUNT(*) FROM faq_questions) as total_questions,
                (SELECT COUNT(*) FROM faq_answers) as total_answers,
                (SELECT COUNT(DISTINCT question_id) FROM faq_answers) as questions_with_answers,
                (SELECT COUNT(*) FROM faq_questions WHERE question_id NOT IN (SELECT question_id FROM faq_answers)) as questions_without_answers
        """, conn).iloc[0]
        
        for key, value in pairing.items():
            print(f"   • {key}: {value:,}")
    else:
        print("\n   ⚠️  No records in faq_answers")

print("✅ faq_answers explored")


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                   TABLE: faq_answers                                                   
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


📊 Table: faq_answers
   Description: FAQ answers linked to questions with version history
   Records: 10
------------------------------------------------------------------------------------------------------------------------

   Schema:
      • answer_id                      INTEGER          [PRIMARY KEY]
      • question_id                    INTEGER          NOT NULL
      • prev_recommended_answer_txt    TEXT            
      • recommended_answer_txt         TEXT            
      • prev_faq_answer_txt            TEXT            
      • faq_answer_txt                 TEXT             NOT NULL
      • user_feedback_txt   

## Step 5: TRACKING TABLES - Change Management

### 5.1 content_change_log

**Purpose**: Single source of truth for all content changes.

**Key Fields**:
- `change_id` - Primary key (auto-increment)
- `content_id` - Foreign key to content_repo
- `change_type` - Type of change (content_edit, position_change, etc.)
- `effective_date` - When change occurred in source
- `detected_at` - When pipeline detected the change

In [49]:
print_header("TABLE: content_change_log", level=2)

with sqlite3.connect(DB_PATH) as conn:
    count = conn.execute("SELECT COUNT(*) FROM content_change_log").fetchone()[0]
    
    print_table_info(
        "content_change_log",
        count,
        "Single source of truth for all content changes"
    )
    
    print_schema(conn, "content_change_log")
    
    if count > 0:
        df = pd.read_sql_query(f"""
            SELECT 
                change_id,
                content_id,
                file_name,
                page_number,
                change_type,
                SUBSTR(content_checksum, 1, 16) as checksum_preview,
                previous_page_number,
                change_trigger,
                detected_at,
                effective_date
            FROM content_change_log
            ORDER BY change_id DESC
            LIMIT {DEFAULT_ROW_LIMIT}
        """, conn)
        
        print("\n📄 Recent Changes:")
        display_dataframe(df)
        
        # Statistics
        print("\n📊 Statistics:")
        stats = pd.read_sql_query("""
            SELECT 
                COUNT(*) as total_changes,
                COUNT(DISTINCT content_id) as unique_content,
                COUNT(DISTINCT file_name) as unique_files,
                MIN(detected_at) as first_detection,
                MAX(detected_at) as last_detection
            FROM content_change_log
        """, conn).iloc[0]
        
        for key, value in stats.items():
            print(f"   • {key}: {value}")
        
        # Change type breakdown
        print("\n\n📊 Changes by Type:")
        types = pd.read_sql_query("""
            SELECT 
                change_type,
                COUNT(*) as count,
                COUNT(*) * 100.0 / (SELECT COUNT(*) FROM content_change_log) as percentage,
                COUNT(DISTINCT file_name) as files_affected
            FROM content_change_log
            GROUP BY change_type
            ORDER BY count DESC
        """, conn)
        display_dataframe(types)
        
        # Recent activity by file
        print("\n\n📂 Recent Activity by File:")
        by_file = pd.read_sql_query("""
            SELECT 
                file_name,
                COUNT(*) as changes,
                COUNT(DISTINCT change_type) as change_types,
                MAX(effective_date) as last_change
            FROM content_change_log
            GROUP BY file_name
            ORDER BY changes DESC
            LIMIT 10
        """, conn)
        display_dataframe(by_file)
    else:
        print("\n   ⚠️  No records in content_change_log")

print("✅ content_change_log explored")


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                               TABLE: content_change_log                                                
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


📊 Table: content_change_log
   Description: Single source of truth for all content changes
   Records: 9
------------------------------------------------------------------------------------------------------------------------

   Schema:
      • change_id                      INTEGER          [PRIMARY KEY]
      • content_id                     INTEGER          NOT NULL
      • file_name                      TEXT             NOT NULL
      • page_number                    INTEGER          NOT NULL
      • content_checksum               TEXT             NOT NULL
      • change_type                    TEXT             NOT NULL
 

### 5.2 faq_content_map

**Purpose**: Links FAQs to content by checksum, managing validity.

**Key Fields**:
- `map_id` - Primary key (auto-increment)
- `question_id`, `answer_id` - Foreign keys to FAQ tables
- `content_checksum` - Links to content version
- `is_valid` - Current validity status
- `invalidation_reason` - Why it was invalidated

In [50]:
print_header("TABLE: faq_content_map", level=2)

with sqlite3.connect(DB_PATH) as conn:
    count = conn.execute("SELECT COUNT(*) FROM faq_content_map").fetchone()[0]
    
    print_table_info(
        "faq_content_map",
        count,
        "Links FAQs to content with validity management"
    )
    
    print_schema(conn, "faq_content_map")
    
    if count > 0:
        df = pd.read_sql_query(f"""
            SELECT 
                map_id,
                question_id,
                answer_id,
                current_file_name,
                current_page_number,
                SUBSTR(content_checksum, 1, 16) as checksum_preview,
                is_valid,
                invalidation_reason,
                confidence_score,
                created_at
            FROM faq_content_map
            ORDER BY map_id DESC
            LIMIT {DEFAULT_ROW_LIMIT}
        """, conn)
        
        print("\n📄 Recent Mappings:")
        display_dataframe(df)
        
        # Statistics
        print("\n📊 Statistics:")
        stats = pd.read_sql_query("""
            SELECT 
                COUNT(*) as total_mappings,
                SUM(CASE WHEN is_valid = 1 THEN 1 ELSE 0 END) as valid_mappings,
                SUM(CASE WHEN is_valid = 0 THEN 1 ELSE 0 END) as invalid_mappings,
                COUNT(DISTINCT question_id) as unique_questions,
                COUNT(DISTINCT content_checksum) as unique_content_versions,
                AVG(confidence_score) as avg_confidence
            FROM faq_content_map
        """, conn).iloc[0]
        
        for key, value in stats.items():
            if isinstance(value, float):
                print(f"   • {key}: {value:.3f}")
            else:
                print(f"   • {key}: {value:,}")
        
        # Validity breakdown
        validity_rate = stats['valid_mappings'] / stats['total_mappings'] * 100 if stats['total_mappings'] > 0 else 0
        print(f"\n   ✅ Validity Rate: {validity_rate:.1f}%")
        
        # Invalidation reasons
        if stats['invalid_mappings'] > 0:
            print("\n\n❌ Invalidation Reasons:")
            reasons = pd.read_sql_query("""
                SELECT 
                    invalidation_reason,
                    COUNT(*) as count,
                    COUNT(*) * 100.0 / (SELECT COUNT(*) FROM faq_content_map WHERE is_valid = 0) as percentage
                FROM faq_content_map
                WHERE is_valid = 0 AND invalidation_reason IS NOT NULL
                GROUP BY invalidation_reason
                ORDER BY count DESC
            """, conn)
            display_dataframe(reasons)
        
        # Mappings by file
        print("\n\n📂 Valid Mappings by File:")
        by_file = pd.read_sql_query("""
            SELECT 
                current_file_name,
                COUNT(*) as total_mappings,
                SUM(CASE WHEN is_valid = 1 THEN 1 ELSE 0 END) as valid_mappings,
                AVG(confidence_score) as avg_confidence
            FROM faq_content_map
            GROUP BY current_file_name
            ORDER BY total_mappings DESC
            LIMIT 10
        """, conn)
        display_dataframe(by_file)
    else:
        print("\n   ⚠️  No records in faq_content_map")

print("✅ faq_content_map explored")


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                 TABLE: faq_content_map                                                 
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


📊 Table: faq_content_map
   Description: Links FAQs to content with validity management
   Records: 0
------------------------------------------------------------------------------------------------------------------------

   Schema:
      • map_id                         INTEGER          [PRIMARY KEY]
      • question_id                    INTEGER          NOT NULL
      • answer_id                      INTEGER         
      • content_checksum               TEXT             NOT NULL
      • current_content_id             INTEGER          NOT NULL
      • current_page_number            INTEGER          NOT NULL
      • curre

### 5.3 content_diffs

**Purpose**: Detailed diffs for content edits with semantic analysis.

**Key Fields**:
- `change_id` - Primary key & foreign key to content_change_log
- `old_checksum`, `new_checksum` - Content versions
- `unified_diff_gzip` - Compressed diff (BLOB)
- `similarity_score` - 0.0 to 1.0 (higher = more similar)
- `semantic_summary` - JSON with semantic changes

In [51]:
print_header("TABLE: content_diffs", level=2)

with sqlite3.connect(DB_PATH) as conn:
    count = conn.execute("SELECT COUNT(*) FROM content_diffs").fetchone()[0]
    
    print_table_info(
        "content_diffs",
        count,
        "Detailed diffs for content edits with semantic analysis"
    )
    
    print_schema(conn, "content_diffs")
    
    if count > 0:
        df = pd.read_sql_query(f"""
            SELECT 
                cd.change_id,
                ccl.file_name,
                ccl.page_number,
                cd.similarity_score,
                cd.chars_added,
                cd.chars_removed,
                cd.lines_added,
                cd.lines_removed,
                cd.is_minor_change,
                cd.processing_time_ms,
                cd.diff_generated_at
            FROM content_diffs cd
            JOIN content_change_log ccl ON cd.change_id = ccl.change_id
            ORDER BY cd.change_id DESC
            LIMIT {DEFAULT_ROW_LIMIT}
        """, conn)
        
        print("\n📄 Recent Diffs:")
        display_dataframe(df)
        
        # Statistics
        print("\n📊 Statistics:")
        stats = pd.read_sql_query("""
            SELECT 
                COUNT(*) as total_diffs,
                AVG(similarity_score) as avg_similarity,
                AVG(chars_added + chars_removed) as avg_chars_changed,
                AVG(lines_added + lines_removed) as avg_lines_changed,
                SUM(CASE WHEN is_minor_change = 1 THEN 1 ELSE 0 END) as minor_changes,
                AVG(processing_time_ms) as avg_processing_time_ms
            FROM content_diffs
        """, conn).iloc[0]
        
        for key, value in stats.items():
            if isinstance(value, float):
                print(f"   • {key}: {value:.2f}")
            else:
                print(f"   • {key}: {value:,}")
        
        # Magnitude distribution
        print("\n\n📊 Change Magnitude Distribution:")
        magnitude = pd.read_sql_query("""
            SELECT 
                CASE 
                    WHEN similarity_score >= 0.9 THEN 'Minor (>90%)'
                    WHEN similarity_score >= 0.7 THEN 'Moderate (70-90%)'
                    WHEN similarity_score >= 0.5 THEN 'Significant (50-70%)'
                    ELSE 'Major (<50%)'
                END as magnitude,
                COUNT(*) as count,
                COUNT(*) * 100.0 / (SELECT COUNT(*) FROM content_diffs) as percentage
            FROM content_diffs
            GROUP BY magnitude
            ORDER BY 
                CASE magnitude
                    WHEN 'Major (<50%)' THEN 1
                    WHEN 'Significant (50-70%)' THEN 2
                    WHEN 'Moderate (70-90%)' THEN 3
                    WHEN 'Minor (>90%)' THEN 4
                END
        """, conn)
        display_dataframe(magnitude)
        
        # Check for semantic summaries
        semantic_count = conn.execute("""
            SELECT COUNT(*) FROM content_diffs WHERE semantic_summary IS NOT NULL
        """).fetchone()[0]
        
        if semantic_count > 0:
            print(f"\n\n🔍 Semantic Analysis Available: {semantic_count} diffs have semantic summaries")
            print("\n   Sample Semantic Summary:")
            sample = conn.execute("""
                SELECT semantic_summary FROM content_diffs 
                WHERE semantic_summary IS NOT NULL 
                LIMIT 1
            """).fetchone()
            if sample:
                try:
                    summary = json.loads(sample[0])
                    print(json.dumps(summary, indent=3))
                except:
                    print("   (Unable to parse semantic summary)")
        else:
            print("\n   ℹ️  No semantic summaries available (requires EnhancedContentDiffProcessor)")
    else:
        print("\n   ⚠️  No records in content_diffs")

print("✅ content_diffs explored")


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                  TABLE: content_diffs                                                  
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


📊 Table: content_diffs
   Description: Detailed diffs for content edits with semantic analysis
   Records: 0
------------------------------------------------------------------------------------------------------------------------

   Schema:
      • change_id                      INTEGER          [PRIMARY KEY]
      • old_checksum                   TEXT             NOT NULL
      • new_checksum                   TEXT             NOT NULL
      • unified_diff_gzip              BLOB             NOT NULL
      • similarity_score               REAL             NOT NULL
      • chars_added                    INTEGER          NOT NU

## Step 6: VIEWS - Derived Data and Insights

### 6.1 v_content_change_summary

**Purpose**: Summary of content changes by file and type.

In [31]:
print_header("VIEW: v_content_change_summary", level=2)

with sqlite3.connect(DB_PATH) as conn:
    try:
        print_view_info(
            "v_content_change_summary",
            "Summary of changes by file and type"
        )
        
        df = pd.read_sql_query(f"""
            SELECT * FROM v_content_change_summary
            ORDER BY change_count DESC
            LIMIT {VIEW_ROW_LIMIT}
        """, conn)
        
        if len(df) > 0:
            print("\n📊 Change Summary by File and Type:")
            display_dataframe(df, max_rows=VIEW_ROW_LIMIT)
        else:
            print("\n   ℹ️  No data in view")
    except Exception as e:
        print(f"\n   ❌ Error querying view: {e}")

print("✅ View explored")


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                             VIEW: v_content_change_summary                                             
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


👁️  View: v_content_change_summary
   Description: Summary of changes by file and type
------------------------------------------------------------------------------------------------------------------------

📊 Change Summary by File and Type:
                file_name      change_type  change_count             first_change              last_change  affected_pages  unique_content_versions
Employee_Leave_Policy.pdf  position_change             4 2025-10-07T11:02:13.000Z 2025-10-07T11:04:08.000Z               4                        3
Employee_Leave_Policy.pdf initial_creation             3 2025-10-07T11:02:13.000Z 2025-10-07T1

### 6.2 v_latest_content_state

**Purpose**: Latest state of each page across all files.

In [32]:
print_header("VIEW: v_latest_content_state", level=2)

with sqlite3.connect(DB_PATH) as conn:
    try:
        print_view_info(
            "v_latest_content_state",
            "Latest state of each page (excludes deletions)"
        )
        
        df = pd.read_sql_query(f"""
            SELECT * FROM v_latest_content_state
            ORDER BY detected_at DESC
            LIMIT {VIEW_ROW_LIMIT}
        """, conn)
        
        if len(df) > 0:
            print("\n📊 Latest Content State:")
            display_dataframe(df, max_rows=VIEW_ROW_LIMIT)
            
            # Summary by file
            print("\n\n📂 Summary by File:")
            summary = pd.read_sql_query("""
                SELECT 
                    file_name,
                    COUNT(*) as total_pages,
                    MAX(last_changed_at) as most_recent_change
                FROM v_latest_content_state
                GROUP BY file_name
                ORDER BY total_pages DESC
            """, conn)
            display_dataframe(summary, max_rows=10)
        else:
            print("\n   ℹ️  No data in view")
    except Exception as e:
        print(f"\n   ❌ Error querying view: {e}")

print("✅ View explored")


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                              VIEW: v_latest_content_state                                              
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


👁️  View: v_latest_content_state
   Description: Latest state of each page (excludes deletions)
------------------------------------------------------------------------------------------------------------------------

📊 Latest Content State:
 content_id                 file_name  page_number                                                 content_checksum last_change_type          last_changed_at          detected_at
       1004 Employee_Leave_Policy.pdf            2 ce9d64d784d0d2618919b6d3a49601785095b21f968c06f71b92a8e1b05be3ef     content_edit 2025-10-07T11:04:08.000Z 2025-10-07T11:04:16Z
       1005 Employee_Leave_Policy.

### 6.3 v_document_structure_changes

**Purpose**: Derived structural changes from position_change patterns (bulk operations).

In [33]:
print_header("VIEW: v_document_structure_changes", level=2)

with sqlite3.connect(DB_PATH) as conn:
    try:
        print_view_info(
            "v_document_structure_changes",
            "Bulk structural changes (3+ pages affected)"
        )
        
        df = pd.read_sql_query(f"""
            SELECT * FROM v_document_structure_changes
            ORDER BY detected_at DESC
            LIMIT {VIEW_ROW_LIMIT}
        """, conn)
        
        if len(df) > 0:
            print("\n📊 Structural Changes Detected:")
            display_dataframe(df, max_rows=VIEW_ROW_LIMIT)
            
            # Summary
            total = len(df)
            total_pages = df['pages_affected'].sum()
            print(f"\n   Total Structural Changes: {total}")
            print(f"   Total Pages Affected: {total_pages:,}")
        else:
            print("\n   ℹ️  No bulk structural changes detected (requires 3+ pages affected)")
    except Exception as e:
        print(f"\n   ❌ Error querying view: {e}")

print("✅ View explored")


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                           VIEW: v_document_structure_changes                                           
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


👁️  View: v_document_structure_changes
   Description: Bulk structural changes (3+ pages affected)
------------------------------------------------------------------------------------------------------------------------

📊 Structural Changes Detected:
 structure_change_id                 file_name change_date     change_type                                                              change_details  pages_affected          detected_at
                   1 Employee_Leave_Policy.pdf  2025-10-07 pages_reordered {"affected_pages":3,"first_page":1,"shift_amount":0,"insertion_point":null}               3 2025-10-07T11:02:47Z
      

### 6.4 v_current_valid_faqs

**Purpose**: Current valid FAQs with change context.

In [34]:
print_header("VIEW: v_current_valid_faqs", level=2)

with sqlite3.connect(DB_PATH) as conn:
    try:
        print_view_info(
            "v_current_valid_faqs",
            "All currently valid FAQs with content context"
        )
        
        df = pd.read_sql_query(f"""
            SELECT 
                map_id,
                question_id,
                answer_id,
                content_file_name,
                content_page_number,
                confidence_score,
                last_content_change,
                content_last_changed
            FROM v_current_valid_faqs
            ORDER BY map_id DESC
            LIMIT {VIEW_ROW_LIMIT}
        """, conn)
        
        if len(df) > 0:
            print("\n📊 Current Valid FAQs:")
            display_dataframe(df, max_rows=VIEW_ROW_LIMIT)
            
            # Count
            total = conn.execute("SELECT COUNT(*) FROM v_current_valid_faqs").fetchone()[0]
            print(f"\n   Total Valid FAQs: {total:,}")
        else:
            print("\n   ⚠️  No valid FAQs found")
    except Exception as e:
        print(f"\n   ❌ Error querying view: {e}")

print("✅ View explored")


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                               VIEW: v_current_valid_faqs                                               
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


👁️  View: v_current_valid_faqs
   Description: All currently valid FAQs with content context
------------------------------------------------------------------------------------------------------------------------

   ⚠️  No valid FAQs found
✅ View explored


### 6.5 v_faq_validity_timeline

**Purpose**: FAQ validity timeline (for audit and reporting).

In [35]:
print_header("VIEW: v_faq_validity_timeline", level=2)

with sqlite3.connect(DB_PATH) as conn:
    try:
        print_view_info(
            "v_faq_validity_timeline",
            "Complete timeline of FAQ validity status"
        )
        
        df = pd.read_sql_query(f"""
            SELECT 
                question_id,
                answer_id,
                current_file_name,
                current_page_number,
                status,
                validity_duration_days,
                invalidation_reason,
                valid_from,
                valid_until
            FROM v_faq_validity_timeline
            ORDER BY created_at DESC
            LIMIT {VIEW_ROW_LIMIT}
        """, conn)
        
        if len(df) > 0:
            print("\n📊 FAQ Validity Timeline:")
            display_dataframe(df, max_rows=VIEW_ROW_LIMIT)
            
            # Summary
            summary = pd.read_sql_query("""
                SELECT 
                    status,
                    COUNT(*) as count,
                    AVG(validity_duration_days) as avg_duration_days
                FROM v_faq_validity_timeline
                GROUP BY status
            """, conn)
            print("\n\n📊 Status Summary:")
            display_dataframe(summary)
        else:
            print("\n   ⚠️  No FAQ validity data")
    except Exception as e:
        print(f"\n   ❌ Error querying view: {e}")

print("✅ View explored")


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                             VIEW: v_faq_validity_timeline                                              
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


👁️  View: v_faq_validity_timeline
   Description: Complete timeline of FAQ validity status
------------------------------------------------------------------------------------------------------------------------

   ⚠️  No FAQ validity data
✅ View explored


### 6.6 v_faqs_needing_review

**Purpose**: Identify FAQs that need review after content changes.

In [36]:
print_header("VIEW: v_faqs_needing_review", level=2)

with sqlite3.connect(DB_PATH) as conn:
    try:
        print_view_info(
            "v_faqs_needing_review",
            "Invalidated FAQs that need review and regeneration"
        )
        
        df = pd.read_sql_query(f"""
            SELECT * FROM v_faqs_needing_review
            ORDER BY days_since_invalidation DESC
            LIMIT {VIEW_ROW_LIMIT}
        """, conn)
        
        if len(df) > 0:
            print("\n⚠️  FAQs Needing Review:")
            display_dataframe(df, max_rows=VIEW_ROW_LIMIT)
            
            total = conn.execute("SELECT COUNT(*) FROM v_faqs_needing_review").fetchone()[0]
            print(f"\n   Total FAQs Needing Review: {total:,}")
            
            if total > 0:
                print("\n   💡 Action Required: Review and regenerate these FAQs")
        else:
            print("\n   ✅ No FAQs currently need review")
    except Exception as e:
        print(f"\n   ❌ Error querying view: {e}")

print("✅ View explored")


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                              VIEW: v_faqs_needing_review                                               
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


👁️  View: v_faqs_needing_review
   Description: Invalidated FAQs that need review and regeneration
------------------------------------------------------------------------------------------------------------------------

   ✅ No FAQs currently need review
✅ View explored


### 6.7 v_content_changes_with_diffs

**Purpose**: Content changes with diff analysis for content edits.

In [37]:
print_header("VIEW: v_content_changes_with_diffs", level=2)

with sqlite3.connect(DB_PATH) as conn:
    try:
        print_view_info(
            "v_content_changes_with_diffs",
            "Content edits with diff metrics and semantic analysis"
        )
        
        df = pd.read_sql_query(f"""
            SELECT 
                change_id,
                file_name,
                page_number,
                similarity_score,
                change_magnitude,
                chars_added,
                chars_removed,
                lines_added,
                lines_removed,
                is_minor_change,
                effective_date
            FROM v_content_changes_with_diffs
            ORDER BY effective_date DESC
            LIMIT {VIEW_ROW_LIMIT}
        """, conn)
        
        if len(df) > 0:
            print("\n📊 Content Changes with Diffs:")
            display_dataframe(df, max_rows=VIEW_ROW_LIMIT)
            
            # Magnitude summary
            print("\n\n📊 Change Magnitude Summary:")
            magnitude = pd.read_sql_query("""
                SELECT 
                    change_magnitude,
                    COUNT(*) as count
                FROM v_content_changes_with_diffs
                WHERE change_magnitude IS NOT NULL
                GROUP BY change_magnitude
            """, conn)
            display_dataframe(magnitude)
        else:
            print("\n   ℹ️  No content changes with diffs yet")
    except Exception as e:
        print(f"\n   ❌ Error querying view: {e}")

print("✅ View explored")


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                           VIEW: v_content_changes_with_diffs                                           
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


👁️  View: v_content_changes_with_diffs
   Description: Content edits with diff metrics and semantic analysis
------------------------------------------------------------------------------------------------------------------------

📊 Content Changes with Diffs:
 change_id                 file_name  page_number similarity_score change_magnitude chars_added chars_removed lines_added lines_removed is_minor_change           effective_date
         7 Employee_Leave_Policy.pdf            2             None            Major        None          None        None          None            None 2025-10-07T11:04:08.000Z
         8 Employee

### 6.8 v_pending_diffs

**Purpose**: Content edits without diffs (processing queue).

In [38]:
print_header("VIEW: v_pending_diffs", level=2)

with sqlite3.connect(DB_PATH) as conn:
    try:
        print_view_info(
            "v_pending_diffs",
            "Content edits awaiting diff generation"
        )
        
        df = pd.read_sql_query(f"""
            SELECT * FROM v_pending_diffs
            ORDER BY days_pending DESC
            LIMIT {VIEW_ROW_LIMIT}
        """, conn)
        
        if len(df) > 0:
            print("\n⚠️  Pending Diffs (oldest first):")
            display_dataframe(df, max_rows=VIEW_ROW_LIMIT)
            
            total = conn.execute("SELECT COUNT(*) FROM v_pending_diffs").fetchone()[0]
            print(f"\n   Total Pending Diffs: {total:,}")
            print("\n   💡 Action Required: Run update_detector.ipynb to generate diffs")
        else:
            print("\n   ✅ No pending diffs - all content edits have been processed")
    except Exception as e:
        print(f"\n   ❌ Error querying view: {e}")

print("✅ View explored")


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                 VIEW: v_pending_diffs                                                  
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


👁️  View: v_pending_diffs
   Description: Content edits awaiting diff generation
------------------------------------------------------------------------------------------------------------------------

⚠️  Pending Diffs (oldest first):
 change_id                 file_name  page_number                                                     new_checksum                                                     old_checksum           effective_date          detected_at  days_pending
         7 Employee_Leave_Policy.pdf            2 ce9d64d784d0d2618919b6d3a49601785095b21f968c06f71b92a8e1b05be3ef b04ed99ce1bf45e408ed7a519c4d4d65554fcccaa7

### 6.9 v_diff_processing_stats

**Purpose**: Summary statistics for diff processing.

In [39]:
print_header("VIEW: v_diff_processing_stats", level=2)

with sqlite3.connect(DB_PATH) as conn:
    try:
        print_view_info(
            "v_diff_processing_stats",
            "Performance metrics for diff generation"
        )
        
        df = pd.read_sql_query("SELECT * FROM v_diff_processing_stats", conn)
        
        if len(df) > 0:
            print("\n📊 Diff Processing Statistics:")
            stats = df.iloc[0]
            print(f"\n   Total Content Edits: {stats['total_content_edits']:,}")
            print(f"   Diffs Generated: {stats['diffs_generated']:,}")
            print(f"   Pending Diffs: {stats['pending_diffs']:,}")
            print(f"   Minor Changes: {stats['minor_changes']:,}")
            print(f"   Major Changes: {stats['major_changes']:,}")
            print(f"   Processing Errors: {stats['errors']:,}")
            
            if stats['diffs_generated'] > 0:
                completion_rate = stats['diffs_generated'] / stats['total_content_edits'] * 100
                print(f"\n   📈 Completion Rate: {completion_rate:.1f}%")
                print(f"   ⏱️  Avg Processing Time: {stats['avg_processing_ms']:.0f} ms")
                print(f"   ⏱️  Max Processing Time: {stats['max_processing_ms']:.0f} ms")
        else:
            print("\n   ℹ️  No diff processing stats available")
    except Exception as e:
        print(f"\n   ❌ Error querying view: {e}")

print("✅ View explored")


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                             VIEW: v_diff_processing_stats                                              
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


👁️  View: v_diff_processing_stats
   Description: Performance metrics for diff generation
------------------------------------------------------------------------------------------------------------------------

📊 Diff Processing Statistics:

   Total Content Edits: 2
   Diffs Generated: 0
   Pending Diffs: 2
   Minor Changes: 0
   Major Changes: 0
   Processing Errors: 0
✅ View explored


## Step 7: Search and Filter

### 7.1 Search by File Name

Find all records related to a specific file across all tables.

In [40]:
print_header("SEARCH BY FILE NAME", level=2)

# Configure search
SEARCH_FILE_NAME = "Employee_Leave_Policy.pdf"  # ⚙️ CHANGE THIS TO YOUR FILE

print(f"🔍 Searching for: {SEARCH_FILE_NAME}\n")

with sqlite3.connect(DB_PATH) as conn:
    # Search in content_repo
    print("📦 content_repo:")
    print("-" * 120)
    content_df = pd.read_sql_query(f"""
        SELECT * FROM content_repo
        WHERE raw_file_nme LIKE '%{SEARCH_FILE_NAME}%'
        ORDER BY raw_file_page_nbr
        LIMIT {SEARCH_LIMIT}
    """, conn)
    
    if len(content_df) > 0:
        print(f"Found {len(content_df)} records")
        display_dataframe(content_df[[
            'ud_source_file_id', 'raw_file_nme', 'raw_file_page_nbr',
            'version_nbr', 'file_status', 'last_modified_dt'
        ]], max_rows=10)
    else:
        print("No records found\n")
    
    # Search in content_change_log
    print("\n🔄 content_change_log:")
    print("-" * 120)
    changes_df = pd.read_sql_query(f"""
        SELECT * FROM content_change_log
        WHERE file_name LIKE '%{SEARCH_FILE_NAME}%'
        ORDER BY detected_at DESC
        LIMIT {SEARCH_LIMIT}
    """, conn)
    
    if len(changes_df) > 0:
        print(f"Found {len(changes_df)} records")
        display_dataframe(changes_df[[
            'change_id', 'file_name', 'page_number', 'change_type',
            'detected_at', 'effective_date'
        ]], max_rows=10)
    else:
        print("No records found\n")
    
    # Search in faq_content_map
    print("\n🗺️  faq_content_map:")
    print("-" * 120)
    faq_map_df = pd.read_sql_query(f"""
        SELECT * FROM faq_content_map
        WHERE current_file_name LIKE '%{SEARCH_FILE_NAME}%'
        ORDER BY current_page_number
        LIMIT {SEARCH_LIMIT}
    """, conn)
    
    if len(faq_map_df) > 0:
        print(f"Found {len(faq_map_df)} records")
        display_dataframe(faq_map_df[[
            'map_id', 'question_id', 'current_file_name', 'current_page_number',
            'is_valid', 'invalidation_reason'
        ]], max_rows=10)
    else:
        print("No records found\n")

print("\n✅ Search complete")


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                  SEARCH BY FILE NAME                                                   
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

🔍 Searching for: Employee_Leave_Policy.pdf

📦 content_repo:
------------------------------------------------------------------------------------------------------------------------
Found 6 records
 ud_source_file_id              raw_file_nme  raw_file_page_nbr  version_nbr file_status         last_modified_dt
              1001 Employee_Leave_Policy.pdf                  1            1      Active 2025-10-07T11:02:13.000Z
              1002 Employee_Leave_Policy.pdf                  2            1      Active 2025-10-07T11:02:13.000Z
              1004 Employee_Leave_Policy.pdf                  2            2      Active 2025-10

### 7.2 Search by Page Number

Find all records for a specific page across all tables.

In [41]:
print_header("SEARCH BY PAGE NUMBER", level=2)

# Configure search
SEARCH_FILE = "Employee_Leave_Policy.pdf"  # ⚙️ CHANGE THIS
SEARCH_PAGE = 2                             # ⚙️ CHANGE THIS

print(f"🔍 Searching for: {SEARCH_FILE}, Page {SEARCH_PAGE}\n")

with sqlite3.connect(DB_PATH) as conn:
    # Get content
    print("📄 Content:")
    print("-" * 120)
    content = pd.read_sql_query("""
        SELECT * FROM content_repo
        WHERE raw_file_nme = ? AND raw_file_page_nbr = ?
    """, conn, params=[SEARCH_FILE, SEARCH_PAGE])
    
    if len(content) > 0:
        display_dataframe(content[[
            'ud_source_file_id', 'raw_file_page_nbr', 'version_nbr',
            'content_checksum', 'file_status', 'last_modified_dt'
        ]])
        
        content_id = content.iloc[0]['ud_source_file_id']
        checksum = content.iloc[0]['content_checksum']
        
        # Get change history
        print("\n📜 Change History:")
        print("-" * 120)
        history = pd.read_sql_query("""
            SELECT * FROM content_change_log
            WHERE file_name = ? AND page_number = ?
            ORDER BY detected_at DESC
        """, conn, params=[SEARCH_FILE, SEARCH_PAGE])
        
        if len(history) > 0:
            display_dataframe(history[[
                'change_id', 'change_type', 'detected_at',
                'effective_date', 'change_trigger'
            ]])
        else:
            print("No change history found\n")
        
        # Get associated FAQs
        print("\n💬 Associated FAQs:")
        print("-" * 120)
        faqs = pd.read_sql_query("""
            SELECT 
                fcm.*,
                SUBSTR(fq.question_txt, 1, 100) as question_preview
            FROM faq_content_map fcm
            LEFT JOIN faq_questions fq ON fcm.question_id = fq.question_id
            WHERE fcm.current_file_name = ? AND fcm.current_page_number = ?
        """, conn, params=[SEARCH_FILE, SEARCH_PAGE])
        
        if len(faqs) > 0:
            display_dataframe(faqs[[
                'map_id', 'question_id', 'is_valid',
                'question_preview', 'confidence_score'
            ]])
        else:
            print("No FAQs associated with this page\n")
    else:
        print(f"\n❌ Page not found: {SEARCH_FILE}, Page {SEARCH_PAGE}")

print("\n✅ Search complete")


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                 SEARCH BY PAGE NUMBER                                                  
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

🔍 Searching for: Employee_Leave_Policy.pdf, Page 2

📄 Content:
------------------------------------------------------------------------------------------------------------------------
 ud_source_file_id  raw_file_page_nbr  version_nbr                                                 content_checksum file_status         last_modified_dt
              1002                  2            1 b04ed99ce1bf45e408ed7a519c4d4d65554fcccaa7e3fbab8ba376393fc24c01      Active 2025-10-07T11:02:13.000Z
              1004                  2            2 ce9d64d784d0d2618919b6d3a49601785095b21f968c06f71b92a8e1b05be3ef      Active 2025-10-07T11:04:

### 7.3 Search by Date Range

Find all changes within a date range.

In [42]:
print_header("SEARCH BY DATE RANGE", level=2)

# Configure search
START_DATE = "2025-01-01"  # ⚙️ CHANGE THIS
END_DATE = "2025-12-31"    # ⚙️ CHANGE THIS

print(f"🔍 Searching for changes between: {START_DATE} and {END_DATE}\n")

with sqlite3.connect(DB_PATH) as conn:
    changes = pd.read_sql_query("""
        SELECT 
            DATE(effective_date) as date,
            change_type,
            COUNT(*) as count,
            COUNT(DISTINCT file_name) as files_affected
        FROM content_change_log
        WHERE DATE(effective_date) BETWEEN ? AND ?
        GROUP BY DATE(effective_date), change_type
        ORDER BY date DESC, count DESC
    """, conn, params=[START_DATE, END_DATE])
    
    if len(changes) > 0:
        print("📊 Changes by Date and Type:")
        display_dataframe(changes, max_rows=30)
        
        # Summary
        total_changes = changes['count'].sum()
        unique_dates = changes['date'].nunique()
        
        print(f"\n📊 Summary:")
        print(f"   Total Changes: {total_changes:,}")
        print(f"   Days with Activity: {unique_dates}")
        print(f"   Avg Changes per Day: {total_changes / unique_dates:.1f}")
    else:
        print("\n❌ No changes found in date range")

print("\n✅ Search complete")


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                  SEARCH BY DATE RANGE                                                  
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

🔍 Searching for changes between: 2025-01-01 and 2025-12-31

📊 Changes by Date and Type:
      date      change_type  count  files_affected
2025-10-07  position_change      4               1
2025-10-07 initial_creation      3               1
2025-10-07     content_edit      2               1

   Showing all 3 rows


📊 Summary:
   Total Changes: 9
   Days with Activity: 1
   Avg Changes per Day: 9.0

✅ Search complete


## Step 8: Data Export

### 8.1 Export Tables to CSV

Export any table or VIEW to CSV for external analysis.

In [43]:
print_header("DATA EXPORT", level=2)

# Configure what to export
EXPORT_TABLES = [
    'content_change_log',
    'faq_content_map',
    'v_current_valid_faqs',
    'v_faqs_needing_review'
]  # ⚙️ CHANGE THIS LIST

print(f"📤 Exporting {len(EXPORT_TABLES)} tables/views to: {EXPORT_DIR}\n")

with sqlite3.connect(DB_PATH) as conn:
    for table_name in EXPORT_TABLES:
        try:
            print(f"\n📊 Exporting {table_name}...")
            
            # Load data
            df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
            
            if len(df) > 0:
                # Generate filename with timestamp
                timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                filename = f"{table_name}_{timestamp}.csv"
                
                # Export
                export_to_csv(df, filename)
                print(f"   Records exported: {len(df):,}")
            else:
                print(f"   ⚠️  No data to export")
        except Exception as e:
            print(f"   ❌ Error exporting {table_name}: {e}")

print("\n\n✅ Export complete")
print(f"\n📁 Exported files are in: {EXPORT_DIR.absolute()}")


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                      DATA EXPORT                                                       
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

📤 Exporting 4 tables/views to: c:\Users\praka\projects\FAQ\FAQ_update\exports


📊 Exporting content_change_log...
   ✅ Exported to: c:\Users\praka\projects\FAQ\FAQ_update\exports\content_change_log_20251007_070425.csv
   Records exported: 9

📊 Exporting faq_content_map...
   ⚠️  No data to export

📊 Exporting v_current_valid_faqs...
   ⚠️  No data to export

📊 Exporting v_faqs_needing_review...
   ⚠️  No data to export


✅ Export complete

📁 Exported files are in: c:\Users\praka\projects\FAQ\FAQ_update\exports


## Step 9: Quick Stats Summary

In [44]:
print_header("QUICK STATS SUMMARY", level=1)

with sqlite3.connect(DB_PATH) as conn:
    print("📊 Database at a Glance:\n")
    
    # All table counts
    tables = [
        'content_repo',
        'faq_questions',
        'faq_answers',
        'content_change_log',
        'faq_content_map',
        'content_diffs'
    ]
    
    print("BASE & TRACKING TABLES:")
    print("-" * 120)
    for table in tables:
        count = conn.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
        print(f"  {table:30} {count:>10,} records")
    
    # Key metrics
    print("\n\nKEY METRICS:")
    print("-" * 120)
    
    # Content metrics
    content_metrics = pd.read_sql_query("""
        SELECT 
            COUNT(DISTINCT raw_file_nme) as unique_files,
            COUNT(*) as total_pages,
            SUM(CASE WHEN file_status = 'Active' THEN 1 ELSE 0 END) as active_pages
        FROM content_repo
    """, conn).iloc[0]
    
    print(f"  Unique Files: {content_metrics['unique_files']:,}")
    print(f"  Total Pages: {content_metrics['total_pages']:,}")
    print(f"  Active Pages: {content_metrics['active_pages']:,}")
    
    # Change metrics
    change_count = conn.execute("SELECT COUNT(*) FROM content_change_log").fetchone()[0]
    if change_count > 0:
        change_metrics = pd.read_sql_query("""
            SELECT 
                COUNT(DISTINCT file_name) as files_with_changes,
                MIN(detected_at) as first_detection,
                MAX(detected_at) as last_detection
            FROM content_change_log
        """, conn).iloc[0]
        
        print(f"  Total Changes: {change_count:,}")
        print(f"  Files with Changes: {change_metrics['files_with_changes']:,}")
    
    # FAQ metrics
    faq_metrics = pd.read_sql_query("""
        SELECT 
            COUNT(*) as total_mappings,
            SUM(CASE WHEN is_valid = 1 THEN 1 ELSE 0 END) as valid_mappings,
            SUM(CASE WHEN is_valid = 0 THEN 1 ELSE 0 END) as invalid_mappings
        FROM faq_content_map
    """, conn).iloc[0]
    
    if faq_metrics['total_mappings'] > 0:
        validity_rate = faq_metrics['valid_mappings'] / faq_metrics['total_mappings'] * 100
        print(f"  Total FAQ Mappings: {faq_metrics['total_mappings']:,}")
        print(f"  Valid FAQs: {faq_metrics['valid_mappings']:,} ({validity_rate:.1f}%)")
        print(f"  Invalid FAQs: {faq_metrics['invalid_mappings']:,}")
    
    # Diff metrics
    diff_count = conn.execute("SELECT COUNT(*) FROM content_diffs").fetchone()[0]
    if diff_count > 0:
        diff_metrics = pd.read_sql_query("""
            SELECT 
                AVG(similarity_score) as avg_similarity,
                SUM(CASE WHEN is_minor_change = 1 THEN 1 ELSE 0 END) as minor_changes
            FROM content_diffs
        """, conn).iloc[0]
        
        print(f"  Total Diffs Generated: {diff_count:,}")
        print(f"  Avg Similarity: {diff_metrics['avg_similarity']:.1%}")
        print(f"  Minor Changes: {diff_metrics['minor_changes']:,}")

print("\n\n" + "=" * 120)
print("✅ CHANGE TRACKER COMPLETE".center(120))
print("=" * 120)
print(f"\n📁 Database: {DB_PATH.absolute()}")
print(f"📁 Exports: {EXPORT_DIR.absolute()}")
print(f"⏰ Completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\n💡 Tip: Adjust DEFAULT_ROW_LIMIT, VIEW_ROW_LIMIT, and SEARCH_LIMIT at the top to see more/fewer rows")
print("=" * 120)


════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════
                                                  QUICK STATS SUMMARY                                                   
════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════

📊 Database at a Glance:

BASE & TRACKING TABLES:
------------------------------------------------------------------------------------------------------------------------
  content_repo                            6 records
  faq_questions                          10 records
  faq_answers                            10 records
  content_change_log                      9 records
  faq_content_map                         0 records
  content_diffs                           0 records


KEY METRICS:
------------------------------------------------------------------------------------------------------------------------
  Unique Files: 1